In [1]:
import pandas as pd
import numpy as np
from pybaseball import statcast
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from pybaseball import playerid_lookup

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Team Postseason Pitching Stats

In [15]:
driver = webdriver.Chrome()
url = "https://www.fangraphs.com/leaders/major-league?pos=all&stats=pit&lg=all&qual=0&type=8&ind=1&startdate=&enddate=&season1=2017&season=2024&postseason=Y&pagenum=1&pageitems=2000000000"
driver.get(url)

In [16]:
html = driver.page_source

In [17]:
soup = BeautifulSoup(html, 'html5lib')

In [18]:
driver.close()

In [19]:
driver.quit()

In [20]:
table = soup.find('div', class_ = 'table-scroll')

In [21]:
columns = []
for i in table.table.thead.find_all('th'):
    try:
        columns.append(i.attrs['data-col-id'].replace('%', '').replace('/', ''))
    except:
        continue
columns = list(filter(lambda a: a != 'divider', columns))
columns.remove('pivFA')
columns.remove('xERA')
columns.remove('WAR')


In [22]:
data_dict = {key:[] for key in columns}

In [23]:
for i in table.table.tbody.find_all('tr'):
    for j in i.find_all('td'):
        try:
            data_dict[j.attrs['data-col-id'].replace('%', '').replace('/', '')].append(j.text.replace('%', ''))
        except:
            None

In [24]:
data_dict

{'Season': ['2019',
  '2019',
  '2021',
  '2020',
  '2017',
  '2020',
  '2023',
  '2019',
  '2017',
  '2023',
  '2023',
  '2020',
  '2022',
  '2023',
  '2023',
  '2020',
  '2023',
  '2020',
  '2021',
  '2023',
  '2018',
  '2021',
  '2020',
  '2021',
  '2018',
  '2017',
  '2022',
  '2022',
  '2019',
  '2022',
  '2023',
  '2018',
  '2017',
  '2023',
  '2018',
  '2022',
  '2017',
  '2021',
  '2018',
  '2022',
  '2019',
  '2017',
  '2024',
  '2024',
  '2017',
  '2022',
  '2020',
  '2020',
  '2017',
  '2020',
  '2017',
  '2021',
  '2019',
  '2018',
  '2019',
  '2023',
  '2019',
  '2021',
  '2024',
  '2023',
  '2023',
  '2021',
  '2023',
  '2024',
  '2018',
  '2024',
  '2024',
  '2019',
  '2021',
  '2023',
  '2021',
  '2018',
  '2020',
  '2024',
  '2020',
  '2022',
  '2022',
  '2024',
  '2019',
  '2018',
  '2022',
  '2021',
  '2018',
  '2021',
  '2024',
  '2024',
  '2021',
  '2019',
  '2019',
  '2024',
  '2018',
  '2023',
  '2021',
  '2022',
  '2022',
  '2018',
  '2022',
  '2022',
  '2020',


In [25]:
data = pd.DataFrame(data_dict)
data = data.loc[data['Season'] != '2020', :]
data.reset_index(drop = True)
#data['BABIP'] = pd.to_numeric(data['BABIP'])
data.drop(columns = ['GS', 'SV'], inplace = True)
data.loc[:, data.columns[7:]] = data.loc[:, data.columns[7:]].apply(pd.to_numeric, axis = 1)
data['LOB'] = data['LOB']/100
data['GB'] = data['GB']/100
data['HRFB'] = data['HRFB']/100

In [26]:
data

,Season,Name,Team,W,L,G,IP,K9,BB9,HR9,BABIP,LOB,GB,HRFB,ERA,FIP,xFIP
0,2019,Gerrit Cole,HOU,4,1,5,36.2,11.54,2.7,0.98,0.224,0.947,0.367,0.108,1.72,2.97,3.56
1,2019,Stephen Strasburg,WSN,5,0,6,36.1,11.64,0.99,0.99,0.299,0.88,0.462,0.154,1.98,2.39,2.38
2,2021,Tyler Matzek,ATL,3,0,13,15.2,13.79,2.3,0.0,0.313,0.786,0.29,0.0,1.72,0.87,2.79
4,2017,Masahiro Tanaka,NYY,2,1,3,20.0,8.1,1.35,0.0,0.196,0.846,0.549,0.0,0.9,1.81,3.14
6,2023,Zack Wheeler,PHI,3,0,5,27.2,11.39,0.98,0.98,0.226,0.823,0.446,0.111,1.95,2.46,2.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,2019,Dakota Hudson,STL,0,1,2,5.0,3.6,5.4,1.8,0.409,0.172,0.478,0.167,9.0,6.81,6.6
1027,2024,Jack Flaherty,LAD,1,2,5,22.0,6.14,3.68,2.45,0.269,0.625,0.417,0.214,7.36,6.71,5.09
1028,2023,Framber Valdez,HOU,0,3,3,12.0,12.75,4.5,3.0,0.455,0.637,0.556,0.364,9.0,6.51,3.69
1030,2017,Mike Montgomery,CHC,0,0,5,4.1,8.31,8.31,6.23,0.524,0.652,0.217,0.273,16.62,13.08,8.6


In [16]:
data.to_csv('postseason_stats.csv', index = False)

## Adding Player IDs

In [5]:
data = pd.read_csv('postseason_stats.csv')

In [6]:
data['ID'] = data['Name'].apply(lambda x: playerid_lookup(last = x.split()[1], first = x.split()[0], fuzzy = True).iloc[0, 2])

Gathering player lookup table. This may take a moment.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identi

In [7]:
data.to_csv('postseason_stats.csv', index = False)

## Adding Metrics

In [8]:
postseason_stats = pd.read_csv('postseason_stats.csv')

In [9]:
postseason_stats['ID'] = postseason_stats['ID'].astype(str)

In [10]:
rs_all_pitch_metrics = None
for year in [2017, 2018, 2019, 2021, 2022, 2023, 2024]:
    print(year)
    raw = pd.read_csv(f'raw_{year}.csv')
    raw['pitcher'] = raw['pitcher'].astype(str)
    rs = raw.loc[raw['game_type'] == 'R', :]
    rs = rs.loc[rs['pitch_name'] != "Pitch Out", :]
    ps = raw.loc[(raw['game_type'] == 'F') | (raw['game_type'] == 'D') | (raw['game_type'] == 'L') | (raw['game_type'] == 'W'), :]
    ps_pitchers = ps['pitcher'].unique()
    rs_ps_pitchers = rs.loc[np.isin(rs['pitcher'], ps_pitchers), ['pitcher', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]
    rs_ps_pitchers_grouped = rs_ps_pitchers.groupby(['pitcher'])
    rs_pitch_metrics = rs_ps_pitchers_grouped.agg('std')
    rs_pitch_metrics = pd.merge(rs_pitch_metrics, rs.loc[np.isin(rs['pitcher'], ps_pitchers), ['pitcher', 'pitch_type']].groupby(['pitcher']).agg('nunique').reset_index().rename(columns = {'pitch_type':'n_pitches'}), how = 'left', on = ['pitcher'])
    rs_pitch_metrics['Season'] = year
    if rs_all_pitch_metrics is None:
        rs_all_pitch_metrics = rs_pitch_metrics
    else:
        rs_all_pitch_metrics = pd.concat([rs_all_pitch_metrics, rs_pitch_metrics])
rs_all_pitch_metrics = rs_all_pitch_metrics.rename(columns = {'pitcher': 'ID'})
data = pd.merge(postseason_stats, rs_all_pitch_metrics, how = 'left', on = ['Season', 'ID'])

2017
2018
2019
2021
2022
2023
2024


C:\Users\John Olsen\AppData\Local\Temp\ipykernel_12004\1899228702.py:4: DtypeWarning: Columns (0,76,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv(f'raw_{year}.csv')


In [11]:
data.shape

(844, 27)

In [12]:
data['release_pos_x'].isna().sum()

np.int64(37)

In [14]:
data = data.dropna()

In [15]:
data.shape

(807, 27)

In [17]:
data.to_csv('all_pitchers_all_pitches.csv', index = False)